In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Экспорт классификатора MNIST в формате SavedModel
В этом упражнении мы узнаем, как создавать модели для TensorFlow Hub. Вам будет поручено выполнить следующие задачи:

* Создание простого классификатора MNIST и оценка его точности.
* Экспорт в SavedModel.
* Размещение модели в качестве модуля TF Hub.
* Импорт этого модуля TF Hub для использования с Keras Layers.


In [1]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

print("\u2022 Используется TensorFlow версия:", tf.__version__)

• Используется TensorFlow версия: 2.9.2


## Создание классификатора MNIST

Мы начнем с создания класса под названием `MNIST`. Этот класс загрузит набор данных MNIST, предварительно обработает изображения из набора данных и построит классификатор на основе CNN. Этот класс также будет иметь несколько методов для обучения, тестирования и сохранения нашей модели.

В ячейке ниже введите недостающий код и создайте следующую модель Keras `Sequential`:

```
    Model: "sequential"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    lambda (Lambda)              (None, 28, 28, 1)         0         
    _________________________________________________________________
    conv2d (Conv2D)              (None, 28, 28, 8)         80        
    _________________________________________________________________
    max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
    _________________________________________________________________
    conv2d_1 (Conv2D)            (None, 14, 14, 16)        1168      
    _________________________________________________________________
    max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
    _________________________________________________________________
    conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
    _________________________________________________________________
    flatten (Flatten)            (None, 1568)              0         
    _________________________________________________________________
    dense (Dense)                (None, 128)               200832    
    _________________________________________________________________
    dense_1 (Dense)              (None, 10)                1290      
    =================================================================

```

Обратите внимание, что мы используем слой `tf.keras.layers.Lambda` в начале нашей модели. Слои `Lambda` используются для обертывания произвольных выражений в виде объекта Layer:

```python
tf.keras.layers.Lambda(expression)
```

The `Lambda` layer exists so that arbitrary TensorFlow functions can be used when constructing `Sequential` and Functional API models. `Lambda` layers are best suited for simple operations. 
Слой `Lambda` существует для того, чтобы произвольные функции `TensorFlow` можно было использовать при построении моделей `Sequential` и  API функций. Слои `Lambda` лучше всего подходят для простых операций.

In [5]:
class MNIST:
    def __init__(self, export_path, buffer_size=1000, batch_size=32,
                 learning_rate=1e-3, epochs=10):
        self._export_path = export_path
        self._buffer_size = buffer_size
        self._batch_size = batch_size
        self._learning_rate = learning_rate
        self._epochs = epochs
    
        self._build_model()
        self.train_dataset, self.test_dataset = self._prepare_dataset()
    
    # функция для предварительной обрабоки изображения
    def preprocess_fn(self, x):
        
        # УПРАЖНЕНИЕ: приведите x к tf.float32 с помощью функции tf.cast ().
        # Вы также должны нормализовать значения x, чтобы они находились в диапазоне [0, 1].
        x = tf.cast(x, tf.float32) / 255.0
            
        return x
        
    def _build_model(self):
        
        # УПРАЖНЕНИЕ: Постройте модель в соответствии с summary модели, показанной выше.
        self._model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(28, 28, 1), dtype=tf.uint8),
            
            # Используйте слой Lambda, чтобы использовать функцию self.preprocess_fn, 
            # определенную выше, для предварительной обработки изображений.
            tf.keras.layers.Lambda(self.preprocess_fn),
            
            # Создайте слой Conv2D с 8 фильтрами, размером ядра 3 и padding='same'.
            tf.keras.layers.Conv2D(filters = 8, kernel_size=3, padding="same"),
            
            # Создайте слой MaxPool2D()со значениями по умалчанию
            tf.keras.layers.MaxPool2D(),
            
            # Создайте слой Conv2D  с 16 фильтрами, размером ядра 3 и padding='same'.
            tf.keras.layers.Conv2D(filters = 16, kernel_size=3, padding="same"),
            
            # Создайте слой MaxPool2D()со значениями по умалчанию
            tf.keras.layers.MaxPool2D(),
            
            # Создайте слой Conv2D  с 32 фильтрами, размером ядра 3 и padding='same'.
            tf.keras.layers.Conv2D(filters = 32, kernel_size=3, padding="same"),
            
            # Создайте слои Flatten и Dense как описано выше в model.summary()
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128),
            tf.keras.layers.Dense(10, activation="softmax")
        ])
        # УПРАЖНЕНИЕ: определите оптимизатор, функцию потерь и метрики.
        # Используйте оптимизатор tf.keras.optimizers.Adam и установите
        # скорость обучения для self._learning_rate.
        optimizer_fn = tf.keras.optimizers.Adam(learning_rate=self._learning_rate)
        
        
        # Используйте sparse_categorical_crossentropy в качестве функции потерь.
        loss_fn = "sparse_categorical_crossentropy"
        
        # установите метрику accuracy.
        metrics_list = ["accuracy"]
     
        # Compile the model.
        self._model.compile(optimizer_fn, loss=loss_fn, metrics=metrics_list)
        
    def _prepare_dataset(self):
        # УПРАЖНЕНИЕ: загрузите набор данных MNIST с помощью tfds.load (). 
        # Вам следует загрузить изображения и также метки, им соответствующие;
        # загружаем как тестовые, так и тренировочные.
        dataset = tfds.load('mnist', split=['train', 'test'], data_dir = f"{os.getcwd()}/../tmp2", as_supervised=True)
        
        
        # УПРАЖНЕНИЕ: разделите ваш датасет на 'train' and 'test'
        train_dataset, test_dataset = dataset
        
        return train_dataset, test_dataset
    
    def train(self):
        # УПРАЖНЕНИЕ: перемешайте и разделите на пакеты набор self.train_dataset. 
        # Используйте self._buffer_size как буфер перемешивания 
        # и self._batch_size как размер пакета
        dataset_tr = self.train_dataset.shuffle(self._buffer_size).batch(self._batch_size)
        
        
        # Тренируйте модель указанное количество эпох
        self._model.fit(dataset_tr, epochs=self._epochs)
        
    def test(self):

        # УПРАЖНЕНИЕ: разделите на пакеты self.test_dataset. Используйте размер пакета 32.
        dataset_te = self.test_dataset.batch(32)
        
        # оценка датасета
        results = self._model.evaluate(dataset_te)
    
        # вывод значений метрики оценки модели.
        for name, value in zip(self._model.metrics_names, results):
            print("%s: %.3f" % (name, value))
            
    def export_model(self):
        # Сохранение модели.
        tf.saved_model.save(self._model, self._export_path)

## Тренировка, оценивание и сохранение  модели

Теперь мы будем использовать созданный выше класс MNIST для создания объекта mnist. При создании нашего объекта `mnist` мы будем использовать словарь для передачи наших параметров обучения. Затем мы вызовем методы `train` и `export_model` для обучения и сохранения нашей модели. Наконец, мы вызовем метод `test`, чтобы оценить нашу модель после обучения.

**ПРИМЕЧАНИЕ:** тренировка модели на 5 эпохах займет около 12 минут.


In [6]:
import os
# Определить параметры тренировки.
args = {'export_path': './saved_model',
        'buffer_size': 1000,
        'batch_size': 32,
        'learning_rate': 1e-3,
        'epochs': 5
}

# Создание объекта mnist. 
mnist = MNIST(**args)

# Тренировка модели.
mnist.train()

# Сохранение модели.
mnist.export_model()

# Оценивание тренированной модели MNIST.
mnist.test()

Dataset mnist downloaded and prepared to /content/../tmp2/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/5
1875/1875 [==============================] - 19s 5ms/step - loss: 0.1528 - accuracy: 0.9542
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0812 - accuracy: 0.9751
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0696 - accuracy: 0.9787
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0632 - accuracy: 0.9806
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0561 - accuracy: 0.9827


313/313 [==============================] - 1s 4ms/step - loss: 0.0680 - accuracy: 0.9792
loss: 0.068
accuracy: 0.979


## Создание файла tarball
Метод `export_model` сохранил нашу модель в формате TensorFlow SavedModel в каталоге `./Saved_model`. Формат SavedModel сохраняет нашу модель и ее веса в различных файлах и каталогах. Так распространять модель будет довольно затруднительно. Поэтому будет удобно создать единый сжатый файл, содержащий все файлы и папки нашей модели. Для этого мы будем использовать программу архивирования tar, чтобы создать tarball (похоже на Zip-файл), содержащий нашу SavedModel.


In [7]:
# Создание a tarball из SavedModel.
!tar -cz -f module.tar.gz -C ./saved_model .

## Проверка Tarball
Мы можем разархивировать наш tarball, чтобы убедиться, что он содерждит все файлы и каталоги нашей сохраненной модели

In [8]:
# Проверка tarball.
!tar -tf module.tar.gz

./
./saved_model.pb
./variables/
./variables/variables.data-00000-of-00001
./variables/variables.index
./assets/


## Моделирование условий сервера

После того, как мы проверили наш архив, мы можем моделировать состояние сервера. В обычном сценарии мы получаем наш модуль TF Hub с удаленного сервера, используя дескриптор модуля. Однако, поскольку этот нутбук не может размещать сервер, вместо этого мы укажем дескриптор модуля на каталог, в котором хранится наша SavedModel.


In [9]:
!rm -rf ./module
!mkdir -p module
!tar xvzf module.tar.gz -C ./module

./
./saved_model.pb
./variables/
./variables/variables.data-00000-of-00001
./variables/variables.index
./assets/


In [10]:
# Определим дескриптор модуля.
MODULE_HANDLE = './module'

## Load the TF Hub Module

In [11]:
# УПРАЖНЕНИЕ: загрузить наш хаб-модуль с использованием hub.load
model = hub.load(MODULE_HANDLE)

## Тестирование модуля TF Hub

Теперь мы протестируем наш модуль TF Hub с изображениями из тестового набора данных MNIST.

In [12]:
# УПРАЖНЕНИЕ: загрузить датасет MNIST, раздел 'test', с использованием tfds.load(). 
# Вам следует загрузить изображения вместе с их метками.

dataset = tfds.load('mnist', split=tfds.Split.TEST, data_dir = f"{os.getcwd()}/../tmp2", as_supervised=True)


# УПРАЖНЕНИЕ: Разделить датасет на пакеты, используя batch size = 32.
test_dataset = dataset.batch(32)

In [13]:
# Тестирование модуля TF Hub на одном пакете данных
for batch_data in test_dataset.take(1):
    outputs = model(batch_data[0])
    outputs = np.argmax(outputs, axis=-1)
    print('Predicted Labels:', outputs)
    print('True Labels:     ', batch_data[1].numpy())

Predicted Labels: [2 0 4 8 7 6 0 6 3 1 8 0 7 9 8 4 5 3 4 0 6 6 3 0 2 3 6 6 2 4 9 3]
True Labels:      [2 0 4 8 7 6 0 6 3 1 8 0 7 9 8 4 5 3 4 0 6 6 3 0 2 3 6 6 7 4 9 3]


Мы видим, что модель правильно предсказывает метки для большинства изображений в пакете.

## Оценка модели с помощью Keras

В ячейке ниже вы интегрируете модуль TensorFlow Hub в высокоуровневый Keras API.

In [14]:
# УПРАЖНЕНИЕ: интегрировать ваш модуль TensorFlow Hub в модель keras.Sequential
# Вам следует использовать hub.KerasLayer и  убедиться, что вы используете
# правильные значения output_shape и input_shape. 
# Вы также должны использовать tf.uint8 для параметра dtype.

model =tf.keras.Sequential([
    hub.KerasLayer(model, output_shape=[10], input_shape=[28,28,1], 
                           dtype=tf.uint8)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [15]:
# Оценка модели на test_dataset.
results = model.evaluate(test_dataset)

313/313 [==============================] - 2s 4ms/step - loss: 0.0680 - accuracy: 0.9792


In [16]:
#Вывод значений метрики оценки
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 0.068
accuracy: 0.979
